In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []

In [3]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2018-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [4]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [5]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2018-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [6]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [7]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])

In [8]:
first_2018 = china_data_export[china_data_export["Period"].str.contains("2018")]

In [9]:
data_2018 = first_2018.groupby(["Comm Code","Commodity"])["YTD Value"].sum()

In [10]:
test= pd.DataFrame({"total" : data_2018})

In [11]:
test

,,total
Comm Code,Commodity,
0101,"HORSES, ASSES, MULES AND HINNIES, LIVE",652308
0102,"BOVINE ANIMALS, LIVE",38176
0103,"SWINE, LIVE",10956584
0104,"SHEEP AND GOATS, LIVE",76920
0105,"CHICKENS, DUCKS, GEESE, TURKEYS, AND GUINEAS, LIVE",151110
0106,"ANIMALS, LIVE, NESOI",3826453
0201,"MEAT OF BOVINE ANIMALS, FRESH OR CHILLED",6225312
0202,"MEAT OF BOVINE ANIMALS, FROZEN",73834008
0203,"MEAT OF SWINE (PORK), FRESH, CHILLED OR FROZEN",230452451


In [12]:
data_2018= test.nlargest(50,"total")


In [13]:
data_2018 = data_2018.reset_index()

In [14]:
data_2018= data_2018.nlargest(50,"total")
data_2018

,Comm Code,Commodity,total
0,8800,"CIVILIAN AIRCRAFT, ENGINES, AND PARTS",15302435584
1,1201,"SOYBEANS, WHETHER OR NOT BROKEN",10928765498
2,8703,MOTOR CARS & VEHICLES FOR TRANSPO,10365709126
3,2709,CRUDE OIL FROM PETROLEUM AND BITUMINOUS MINERALS,6358994062
4,8542,"ELECTRONIC INTEGRATED CIRCUITS & MICROASSEMBL,...",4659296273
5,8486,MACH/APPS FOR MANUFCT OF SEMIC,4106049218
6,9018,"MEDICAL, SURGICAL, DENTAL",3082454988
7,8708,PARTS & ACCESS FOR MOTOR,2925948928
8,2711,PETROLEUM GASES & OTHER GASEOUS HYDROCARBONS,2135501692
9,5201,"COTTON, NOT CARDED OR COMBED",2093837426


In [15]:
data_2018 = data_2018.to_csv("data_2018.csv", index=False)